In [2]:
import torch
import torchvision.models as models
from torch.profiler import profile, record_function, ProfilerActivity

In [3]:
model = models.resnet18()
inputs = torch.randn(5, 3, 224, 224)

In [4]:
with profile(activities=[ProfilerActivity.CPU], record_shapes=True) as prof:
    with record_function("model_inference"):
        model(inputs)

STAGE:2024-06-05 22:45:09 33:33 ActivityProfilerController.cpp:312] Completed Stage: Warm Up
STAGE:2024-06-05 22:45:10 33:33 ActivityProfilerController.cpp:318] Completed Stage: Collection
STAGE:2024-06-05 22:45:10 33:33 ActivityProfilerController.cpp:322] Completed Stage: Post Processing


In [5]:
print(prof.key_averages().table(sort_by="cpu_time_total", row_limit=10))

---------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  
                             Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg    # of Calls  
---------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  
                  model_inference         4.00%      23.812ms       100.00%     595.622ms     595.622ms             1  
                     aten::conv2d         0.60%       3.601ms        71.80%     427.647ms      21.382ms            20  
                aten::convolution         0.32%       1.900ms        71.19%     424.046ms      21.202ms            20  
               aten::_convolution         0.47%       2.806ms        70.87%     422.146ms      21.107ms            20  
         aten::mkldnn_convolution        70.09%     417.497ms        70.40%     419.340ms      20.967ms            20  
                 aten::batch_norm       

In [6]:
print(prof.key_averages().table(sort_by="cpu_memory_usage", row_limit=10))

---------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  
                             Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg    # of Calls  
---------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  
                  model_inference         4.00%      23.812ms       100.00%     595.622ms     595.622ms             1  
                     aten::conv2d         0.60%       3.601ms        71.80%     427.647ms      21.382ms            20  
                aten::convolution         0.32%       1.900ms        71.19%     424.046ms      21.202ms            20  
               aten::_convolution         0.47%       2.806ms        70.87%     422.146ms      21.107ms            20  
         aten::mkldnn_convolution        70.09%     417.497ms        70.40%     419.340ms      20.967ms            20  
                      aten::empty       

In [7]:
print(prof.key_averages().table(sort_by="self_cpu_memory_usage", row_limit=10))

---------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  
                             Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg    # of Calls  
---------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  
                  model_inference         4.00%      23.812ms       100.00%     595.622ms     595.622ms             1  
                     aten::conv2d         0.60%       3.601ms        71.80%     427.647ms      21.382ms            20  
                aten::convolution         0.32%       1.900ms        71.19%     424.046ms      21.202ms            20  
               aten::_convolution         0.47%       2.806ms        70.87%     422.146ms      21.107ms            20  
         aten::mkldnn_convolution        70.09%     417.497ms        70.40%     419.340ms      20.967ms            20  
                      aten::empty       

In [10]:
print(prof.key_averages())

---------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  
                             Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg    # of Calls  
---------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  
                  model_inference         4.00%      23.812ms       100.00%     595.622ms     595.622ms             1  
                     aten::conv2d         0.60%       3.601ms        71.80%     427.647ms      21.382ms            20  
                aten::convolution         0.32%       1.900ms        71.19%     424.046ms      21.202ms            20  
               aten::_convolution         0.47%       2.806ms        70.87%     422.146ms      21.107ms            20  
         aten::mkldnn_convolution        70.09%     417.497ms        70.40%     419.340ms      20.967ms            20  
                      aten::empty       

In [12]:
with profile(activities=[ProfilerActivity.CPU],
        profile_memory=True, record_shapes=True) as prof:
    model(inputs)

print(prof.key_averages())


STAGE:2024-06-05 22:50:02 33:33 ActivityProfilerController.cpp:312] Completed Stage: Warm Up


---------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                             Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg       CPU Mem  Self CPU Mem    # of Calls  
---------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                     aten::conv2d         0.07%     179.000us        72.52%     192.546ms       9.627ms      47.37 Mb           0 b            20  
                aten::convolution         0.19%     503.000us        72.45%     192.367ms       9.618ms      47.37 Mb           0 b            20  
               aten::_convolution         0.12%     310.000us        72.26%     191.864ms       9.593ms      47.37 Mb           0 b            20  
         aten::mkldnn_convolution        71.95%     191.038ms        72.14%     191.554ms       9.578ms      47.

STAGE:2024-06-05 22:50:02 33:33 ActivityProfilerController.cpp:318] Completed Stage: Collection
STAGE:2024-06-05 22:50:02 33:33 ActivityProfilerController.cpp:322] Completed Stage: Post Processing


In [5]:
print(prof.key_averages(group_by_input_shape=True).table(sort_by="cpu_time_total", row_limit=10))

---------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  --------------------------------------------------------------------------------  
                             Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg    # of Calls                                                                      Input Shapes  
---------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  --------------------------------------------------------------------------------  
                  model_inference         4.75%      21.873ms       100.00%     460.041ms     460.041ms             1                                                                                []  
                     aten::conv2d         0.72%       3.295ms        20.67%      95.109ms      95.109ms             1                             [[5, 3, 224, 224], [64, 3, 7, 7], [], [], [], 

In [8]:
model = models.resnet18().cuda()
inputs = torch.randn(5, 3, 224, 224).cuda()

with profile(activities=[
        ProfilerActivity.CPU, ProfilerActivity.CUDA], record_shapes=True) as prof:
    with record_function("model_inference"):
        model(inputs)

print(prof.key_averages().table(sort_by="cuda_time_total", row_limit=10))

-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                                   Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA total  CUDA time avg    # of Calls  
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                        model_inference        20.50%       2.302ms        73.16%       8.214ms       8.214ms       0.000us         0.00%       9.912ms       9.912ms             1  
                                           aten::conv2d         0.69%      78.000us        22.68%       2.547ms     127.350us       0.000us         0.00%       8.142ms     407.100us            20  
         

STAGE:2024-06-05 21:34:58 34:34 ActivityProfilerController.cpp:312] Completed Stage: Warm Up
STAGE:2024-06-05 21:34:58 34:34 ActivityProfilerController.cpp:318] Completed Stage: Collection
STAGE:2024-06-05 21:34:58 34:34 ActivityProfilerController.cpp:322] Completed Stage: Post Processing


In [9]:
with open('print_to_file.csv', 'w') as f:
    print(prof.key_averages().table(sort_by="cuda_time_total", row_limit=10),file=f)

In [5]:
model = models.resnet18()
inputs = torch.randn(5, 3, 224, 224)

with profile(activities=[ProfilerActivity.CPU],
        profile_memory=True, record_shapes=True) as prof:
    model(inputs)

print(prof.key_averages().table(sort_by="self_cpu_memory_usage", row_limit=10))


STAGE:2024-06-05 21:30:08 34:34 ActivityProfilerController.cpp:312] Completed Stage: Warm Up
STAGE:2024-06-05 21:30:08 34:34 ActivityProfilerController.cpp:318] Completed Stage: Collection
STAGE:2024-06-05 21:30:08 34:34 ActivityProfilerController.cpp:322] Completed Stage: Post Processing


---------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                             Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg       CPU Mem  Self CPU Mem    # of Calls  
---------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                      aten::empty         0.26%     387.000us         0.26%     387.000us       1.935us      94.36 Mb      94.36 Mb           200  
    aten::max_pool2d_with_indices        16.15%      24.071ms        16.15%      24.071ms      24.071ms      11.48 Mb      11.48 Mb             1  
                 aten::empty_like         0.04%      62.000us         0.07%     104.000us       5.200us      47.37 Mb     490.00 Kb            20  
                      aten::addmm         0.21%     308.000us         0.22%     323.000us     323.000us      19.